# Data Wrangling

## Spark Loading

In [3]:
%load_ext sparkmagic.magics

TypeError: options must be a list of values or a list of (label, value) tuples

In [ ]:
import pickle
path_data = "../data/"

In [ ]:
# pull from lfs
! git lfs pull

In [ ]:
import os
from IPython import get_ipython
username = os.environ['RENKU_USERNAME']
server = "http://iccluster029.iccluster.epfl.ch:8998"

# set the application name as "<your_gaspar_id>-homework3"
get_ipython().run_cell_magic(
    'spark',
    line='config', 
    cell="""{{ "name": "{0}-homework3", "executorMemory": "4G", "executorCores": 4, "numExecutors": 10, "driverMemory": "4G"}}""".format(username)
)

In [2]:
get_ipython().run_line_magic(
    "spark", "add -s {0}-homework3 -l python -u {1} -k".format(username, server)
)

NameError: name 'username' is not defined

In [6]:
%%spark
print('We are using Spark %s' % spark.version)

An error was encountered:
You need to have at least 1 client created to execute commands.


## Stops 15km from Zürich HB (8503000) (lat, lon) = (47.378177, 8.540192)

In [7]:
%%spark
all_stops = spark.read.csv('/data/sbb/csv/allstops/stop_locations.csv')
all_stops.show(10)

An error was encountered:
You need to have at least 1 client created to execute commands.


In [8]:
%%spark
all_stops = all_stops.withColumnRenamed('_c0','stop_id')
all_stops = all_stops.withColumnRenamed('_c1','stop_name')
all_stops = all_stops.withColumnRenamed('_c2','lat')
all_stops = all_stops.withColumnRenamed('_c3','lon')
all_stops = all_stops.drop('_c4','_c5')
all_stops.show(10)

An error was encountered:
You need to have at least 1 client created to execute commands.


In [9]:
%%spark
import pyspark.sql.functions as F
zur_hp_lat = 47.378177
zur_hp_lon = 8.540192

all_stops_distance = all_stops.withColumn("a", (
        F.pow(F.sin(F.radians(zur_hp_lat - F.col("lat")) / 2), 2) +
        F.cos(F.radians(F.col("lat"))) * F.cos(F.radians(F.lit(zur_hp_lat))) *
        F.pow(F.sin(F.radians(zur_hp_lon - F.col("lon")) / 2), 2)
    )).withColumn("distance", F.atan2(F.sqrt(F.col("a")), F.sqrt(-F.col("a") + 1)) * 12742000)

all_stops_distance.show(10)

An error was encountered:
You need to have at least 1 client created to execute commands.


In [10]:
%%spark
close_stops = all_stops_distance.filter(all_stops_distance['distance'] <= 15000).drop('a','distance')
close_stops.show(10)

An error was encountered:
You need to have at least 1 client created to execute commands.


In [11]:
%%spark
close_stops.count()

An error was encountered:
You need to have at least 1 client created to execute commands.


## Weekly timetable

Considered period: May 08-14, 2019 (then we exclude the weekends)

In [12]:
%%spark
all_trips1 = spark.read.csv('/data/sbb/csv/stop_times/2019/05/08/stop_times.txt', header=True)
#all_trips2 = spark.read.csv('/data/sbb/csv/stop_times/2019/05/15/stop_times.txt', header=True)

An error was encountered:
You need to have at least 1 client created to execute commands.


In [13]:
%%spark
all_trips1.count()

An error was encountered:
You need to have at least 1 client created to execute commands.


In [14]:
%%spark
#all_trips2.count()

An error was encountered:
You need to have at least 1 client created to execute commands.


In [15]:
%%spark
all_trips1.filter(all_trips1.arrival_time==all_trips1.departure_time).count()

An error was encountered:
You need to have at least 1 client created to execute commands.


In [16]:
%%spark
#all_trips_tot = all_trips1.union(all_trips2)
all_trips_tot = all_trips1
all_trips_tot.show(10)

An error was encountered:
You need to have at least 1 client created to execute commands.


In [17]:
%%spark
routes1 = spark.read.csv('/data/sbb/csv/trips/2019/05/08/trips.txt', header=True)
#routes2 = spark.read.csv('/data/sbb/csv/trips/2019/05/15/trips.txt', header=True)
#routes_tot = routes1.union(routes2)
routes_tot = routes1
routes_tot.show(10)

An error was encountered:
You need to have at least 1 client created to execute commands.


In [18]:
%%spark
trips_with_routes = all_trips_tot.join(routes_tot, all_trips_tot.trip_id == routes_tot.trip_id)
trips_with_routes.show(10)

An error was encountered:
You need to have at least 1 client created to execute commands.


In [19]:
%%spark
trips_with_routes = trips_with_routes.select(all_trips_tot.trip_id, 'service_id').distinct()
trips_with_routes.show(10)

An error was encountered:
You need to have at least 1 client created to execute commands.


In [20]:
%%spark
cal1 = spark.read.csv('/data/sbb/csv/calendar/2019/05/08/calendar.txt', header=True)
#cal2 = spark.read.csv('/data/sbb/csv/calendar/2019/05/15/calendar.txt', header=True)
#cal_tot = cal1.union(cal2).distinct()
cal_tot = cal1.distinct()
cal_tot.show(10)

An error was encountered:
You need to have at least 1 client created to execute commands.


In [21]:
%%spark
cal_filtered = cal_tot.filter((cal_tot.monday == 1) | (cal_tot.tuesday == 1) | (cal_tot.wednesday == 1) | (cal_tot.thursday == 1) | (cal_tot.friday == 1))
cal_filtered_dates = cal_filtered.filter((cal_tot.start_date <= '20190513') & (cal_tot.end_date >= '20190518'))
cal_filtered_dates.show(10)

An error was encountered:
You need to have at least 1 client created to execute commands.


In [22]:
%%spark
service_ids = cal_filtered_dates.drop('start_date', 'end_date', 'saturday', 'sunday')

An error was encountered:
You need to have at least 1 client created to execute commands.


In [23]:
%%spark
service_ids.select('service_id')
trips_in_period = trips_with_routes.join(service_ids, service_ids.service_id == trips_with_routes.service_id,'inner').drop(trips_with_routes.service_id).distinct()
trips_in_period.show(10)

An error was encountered:
You need to have at least 1 client created to execute commands.


In [24]:
%%spark
final_trips = all_trips_tot.join(trips_in_period, trips_in_period.trip_id == all_trips_tot.trip_id).distinct().drop(all_trips_tot.trip_id)
final_trips.show(10)

An error was encountered:
You need to have at least 1 client created to execute commands.


In [25]:
%%spark
all_trips_tot.count()

An error was encountered:
You need to have at least 1 client created to execute commands.


In [26]:
%%spark
final_trips.count()

An error was encountered:
You need to have at least 1 client created to execute commands.


### Considering working hours from 9:00 to 16:59

In [27]:
%%spark
import pyspark.sql.functions as F

final_trips = final_trips.filter(F.hour(final_trips.arrival_time).between(9,17) & F.hour(final_trips.departure_time).between(9,17))
final_trips.show(10, truncate=False)

An error was encountered:
You need to have at least 1 client created to execute commands.


In [28]:
%%spark
final_trips.count()

An error was encountered:
You need to have at least 1 client created to execute commands.


### Keeping only trip_ids that contains a stop in the area of Zurich

In [29]:
%%spark
final_close_trips_ids = final_trips.join(close_stops, final_trips.stop_id == close_stops.stop_id , 'inner').select('trip_id').distinct()
final_close_trips_ids.show(10)

An error was encountered:
You need to have at least 1 client created to execute commands.


In [30]:
%%spark
final_close_trips1 = final_trips.join(final_close_trips_ids, final_trips.trip_id == final_close_trips_ids.trip_id).drop(final_close_trips_ids.trip_id).distinct()
final_close_trips2 = final_close_trips1.join(all_stops_distance, all_stops_distance.stop_id == final_close_trips1.stop_id).drop(final_close_trips1.stop_id).distinct()
final_close_trips_filtered = final_close_trips2.filter(all_stops_distance['distance'] <= 18000).drop('a','distance', 'lat', 'lon', 'stop_name')
final_close_trips_filtered.show()

An error was encountered:
You need to have at least 1 client created to execute commands.


In [31]:
%%spark
final_close_trips_filtered.cache()
final_close_trips_filtered.count()

An error was encountered:
You need to have at least 1 client created to execute commands.


### Making table to link type of mean of transport to trip_ids

In [ ]:
%%spark
trips_id = final_close_trips_filtered.select("trip_id").distinct()
trips = spark.read.csv('/data/sbb/csv/trips/2019/05/08/trips.txt', header=True)

trips_filtered = trips.join(trips_id, "trip_id", "inner")

trips_filtered = trips_filtered.select("trip_id", "route_id")

routes = spark.read.csv('/data/sbb/csv/routes/2019/05/08/routes.txt', header=True)
routes_descs = routes.select("route_desc", "route_id")

joined_routes = routes_descs.join(trips_filtered, "route_id", "inner")

joined_routes = joined_routes.select("trip_id", "route_desc")

joined_routes.show(10)


In [ ]:
%%spark
#joined_routes.write.mode('overwrite').option("delimiter", ";").option("header","true").format("csv").save("/group/five-guys/routes")

## Create timetable

As request by the paper 

### stops Table

In [32]:
%%spark
stops_table = final_close_trips_filtered.select(final_close_trips_filtered.stop_id).distinct()
stops_table.show(10)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------+
|    stop_id|
+-----------+
|    8503078|
|    8506895|
|    8589111|
|    8591284|
|    8590819|
|    8573729|
|    8591190|
|    8502508|
|    8503376|
|8503306:0:2|
+-----------+
only showing top 10 rows

In [33]:
%%spark

all_stops = spark.read.csv('/data/sbb/csv/allstops/stop_locations.csv')

all_stops = all_stops.withColumnRenamed('_c0','stop_id')
all_stops = all_stops.withColumnRenamed('_c1','stop_name')
all_stops = all_stops.withColumnRenamed('_c2','stop_lat')
all_stops = all_stops.withColumnRenamed('_c3','stop_lon')
all_stops = all_stops.withColumnRenamed('_c4','location_type')
all_stops = all_stops.withColumnRenamed('_c5','parent_station')
all_stops = all_stops.drop('location_type')

all_stops_full = all_stops.join(stops_table, "stop_id" ,"inner")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [34]:
%%spark
all_stops_full.show(10)
#all_stops_full.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+--------------------+----------------+----------------+--------------+
|stop_id|           stop_name|        stop_lat|        stop_lon|parent_station|
+-------+--------------------+----------------+----------------+--------------+
|1100150|Schlächtenhaus, S...|47.6898550981112|7.74233688977719|          null|
|1100376|Brombach (LÖ), Fr...|47.6314693778999|7.68696473565635|          null|
|1100411|Häg-Ehrsberg, Ehr...|  47.74319213737|7.90619858077624|          null|
|1101439|   Gailingen, Schule|47.6948133619046|8.75378600091951|          null|
|1101511|Hoppetenzell, Sar...|47.8931911144564|9.01140485813517|          null|
|1101517|           Igelswies|48.0050619291514|9.13811222897787|          null|
|1101664|  Liptingen, Rathaus|47.9384005993389|8.90860165700622|          null|
|1102913|Lindau (D), Westl...|47.5459998093133|9.67818836101854|          null|
|1103354|Tettnang, Bärenplatz|47.6711364278945|9.59007261478699|          null|
|1103389|Tettnang, Gut Kal...|47.6844488

In [35]:
%%spark
#all_stops_full.write.mode('overwrite').option("delimiter", ";").option("header","true").format("csv").save("/group/five-guys/stops_table")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### connections Table

In [36]:
%%spark
trip_ids_with_days_table = final_close_trips_filtered.select('trip_id', 'monday', 'tuesday', 'wednesday', 'thursday', 'friday').distinct()
trip_ids_with_days_table.show(10)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+------+-------+---------+--------+------+
|             trip_id|monday|tuesday|wednesday|thursday|friday|
+--------------------+------+-------+---------+--------+------+
|1.TA.26-18-j19-1.1.H|     1|      1|        1|       1|     1|
|1.TA.63-138-Y-j19...|     0|      0|        1|       0|     0|
|1001.TA.26-213-j1...|     0|      0|        0|       0|     1|
|1014.TA.26-520-j1...|     1|      1|        1|       1|     0|
|1017.TA.26-151-j1...|     1|      1|        1|       1|     0|
|1023.TA.26-4-j19-...|     1|      1|        1|       1|     1|
|1029.TA.26-LAF-j1...|     1|      1|        1|       1|     0|
|103.TA.26-510-j19...|     0|      0|        0|       0|     1|
|1039.TA.26-200-j1...|     0|      0|        0|       0|     1|
|1082.TA.26-201-j1...|     0|      0|        0|       0|     1|
+--------------------+------+-------+---------+--------+------+
only showing top 10 rows

In [37]:
%%spark
working_table = final_close_trips_filtered.withColumn('stop_sequence',final_close_trips_filtered.stop_sequence.cast('int')).orderBy('trip_id', 'stop_sequence').drop('service_id', 'pickup_type', 'drop_off_type', 'monday',  'tuesday', 'thursday', 'wednesday', 'thursday', 'friday')
working_table = working_table.withColumnRenamed('stop_id','departure_id')
working_table = working_table.withColumnRenamed('arrival_time','old_arrival_time')
working_table.show(30)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------------+--------------+-------------+--------------------+------------+
|old_arrival_time|departure_time|stop_sequence|             trip_id|departure_id|
+----------------+--------------+-------------+--------------------+------------+
|        15:10:00|      15:10:00|            1|1.TA.1-245-j19-1.1.H|     8573710|
|        15:11:00|      15:11:00|            2|1.TA.1-245-j19-1.1.H|     8591341|
|        15:13:00|      15:13:00|            3|1.TA.1-245-j19-1.1.H|     8502572|
|        15:15:00|      15:15:00|            4|1.TA.1-245-j19-1.1.H|     8580912|
|        15:17:00|      15:17:00|            5|1.TA.1-245-j19-1.1.H|     8503610|
|        15:18:00|      15:18:00|            6|1.TA.1-245-j19-1.1.H|     8573711|
|        15:21:00|      15:21:00|            7|1.TA.1-245-j19-1.1.H|     8503709|
|        15:23:00|      15:23:00|            8|1.TA.1-245-j19-1.1.H|     8503699|
|        15:24:00|      15:24:00|            9|1.TA.1-245-j19-1.1.H|     8503580|
|        15:25:0

In [38]:
%%spark
import pyspark.sql.functions as F
from pyspark.sql.window import Window

w = Window().partitionBy('trip_id').orderBy('stop_sequence')
table_a = working_table.select("*",F.lag("old_arrival_time", -1).over(w).alias("arrival_time"), F.lag("departure_id", -1).over(w).alias("arrival_id")).na.drop()
table_a.show(10)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------------+--------------+-------------+--------------------+------------+------------+-----------+
|old_arrival_time|departure_time|stop_sequence|             trip_id|departure_id|arrival_time| arrival_id|
+----------------+--------------+-------------+--------------------+------------+------------+-----------+
|        10:41:00|      10:41:00|            1|1.TA.26-18-j19-1.1.H|     8503064|    10:45:00|8503065:0:1|
|        10:45:00|      10:45:00|            2|1.TA.26-18-j19-1.1.H| 8503065:0:1|    10:46:00|    8503074|
|        10:46:00|      10:46:00|            3|1.TA.26-18-j19-1.1.H|     8503074|    10:47:00|    8503068|
|        10:47:00|      10:47:00|            4|1.TA.26-18-j19-1.1.H|     8503068|    10:48:00|    8503066|
|        10:48:00|      10:48:00|            5|1.TA.26-18-j19-1.1.H|     8503066|    10:50:00|    8503075|
|        10:50:00|      10:50:00|            6|1.TA.26-18-j19-1.1.H|     8503075|    10:52:00|    8503067|
|        10:52:00|      10:52:00|    

In [39]:
%%spark
import pyspark.sql.functions as F
from pyspark.sql.window import Window

final_table = table_a.join(trip_ids_with_days_table, table_a.trip_id == trip_ids_with_days_table.trip_id)
connections_table = final_table.drop('old_arrival_time', 'stop_sequence').select('departure_id', 'arrival_id', 'departure_time', 'arrival_time', table_a.trip_id, 'monday', 'tuesday', 'wednesday', 'thursday', 'friday')
connections_table.show(10)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------+----------+--------------+------------+--------------------+------+-------+---------+--------+------+
|departure_id|arrival_id|departure_time|arrival_time|             trip_id|monday|tuesday|wednesday|thursday|friday|
+------------+----------+--------------+------------+--------------------+------+-------+---------+--------+------+
|     8503082|   8503081|      13:20:00|    13:25:00|1029.TA.26-LAF-j1...|     1|      1|        1|       1|     0|
|     8591023|   8591028|      16:46:00|    16:49:00|120.TA.26-91-j19-...|     1|      1|        1|       1|     1|
|     8591903|   8591023|      16:45:00|    16:46:00|120.TA.26-91-j19-...|     1|      1|        1|       1|     1|
|     8591024|   8591903|      16:44:00|    16:45:00|120.TA.26-91-j19-...|     1|      1|        1|       1|     1|
|     8591029|   8591024|      16:44:00|    16:44:00|120.TA.26-91-j19-...|     1|      1|        1|       1|     1|
|     8591400|   8591029|      16:43:00|    16:44:00|120.TA.26-91-j19-..

In [40]:
%%spark
connections_table.filter(connections_table.arrival_time < connections_table.departure_time).count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

0

In [41]:
%%spark
connections_table.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

495470

## Full table

In [42]:
%%spark
from pyspark.sql.functions import desc
connections_table = connections_table.orderBy(desc("departure_time"))
connections_table.show(10)
#ordering of columns
#connections_table = connections_table_monday_full_sorted.select("departure_id", "arrival_id", "departure_time", "arrival_time", "trip_id")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------+----------+--------------+------------+--------------------+------+-------+---------+--------+------+
|departure_id|arrival_id|departure_time|arrival_time|             trip_id|monday|tuesday|wednesday|thursday|friday|
+------------+----------+--------------+------------+--------------------+------+-------+---------+--------+------+
|     8502879|   8570706|      17:59:00|    17:59:00|1198.TA.26-215-j1...|     0|      0|        0|       0|     1|
|     8503690|   8583053|      17:59:00|    17:59:00|234.TA.26-236-j19...|     0|      0|        0|       0|     1|
|     8502879|   8570706|      17:59:00|    17:59:00|1415.TA.26-215-j1...|     1|      1|        1|       1|     0|
|     8503709|   8574684|      17:59:00|    17:59:00|179.TA.26-201-j19...|     0|      0|        0|       0|     1|
|     8591333|   8591116|      17:59:00|    17:59:00|482.TA.26-78-j19-...|     1|      1|        1|       1|     1|
|     8503709|   8574684|      17:59:00|    17:59:00|585.TA.26-201-j19..

In [43]:
%%spark
#connections_table.write.mode('overwrite').option("header","true").format("csv").save("/group/five-guys/conn_table")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### trips Table

In [44]:
%%spark
#final_close_trips_filtered = final_close_trips.filter(final_close_trips.arrival_time>final_close_trips.departure_time)
trips_table = final_close_trips_filtered.select(final_close_trips_filtered.trip_id).distinct()
trips_table.show(10)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+
|             trip_id|
+--------------------+
|1.TA.26-18-j19-1.1.H|
|1.TA.63-138-Y-j19...|
|1001.TA.26-213-j1...|
|1014.TA.26-520-j1...|
|1017.TA.26-151-j1...|
|1023.TA.26-4-j19-...|
|1029.TA.26-LAF-j1...|
|103.TA.26-510-j19...|
|1039.TA.26-200-j1...|
|1082.TA.26-201-j1...|
+--------------------+
only showing top 10 rows

In [45]:
%%spark 
#trips as described in the readme, but filtered for time and distance

trips_table_filtered = routes_tot.join(trips_table, "trip_id" ,"inner")
trips_table_filtered.show(10)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+-----------+----------+--------------------+---------------+------------+
|             trip_id|   route_id|service_id|       trip_headsign|trip_short_name|direction_id|
+--------------------+-----------+----------+--------------------+---------------+------------+
|9.TA.1-217-j19-1.1.H|1-217-j19-1|  TA+b0001|Affoltern a. A., ...|          21719|           0|
|15.TA.1-217-j19-1...|1-217-j19-1|  TA+b0001|Affoltern a. A., ...|          21731|           0|
|17.TA.1-217-j19-1...|1-217-j19-1|  TA+b0001|Affoltern a. A., ...|          21735|           0|
|19.TA.1-217-j19-1...|1-217-j19-1|  TA+b0001|Affoltern a. A., ...|          21739|           0|
|21.TA.1-217-j19-1...|1-217-j19-1|  TA+b0001|Affoltern a. A., ...|          21743|           0|
|23.TA.1-217-j19-1...|1-217-j19-1|  TA+b0001|Affoltern a. A., ...|          21747|           0|
|11.TA.1-217-j19-1...|1-217-j19-1|  TA+b0001|Affoltern a. A., ...|          21723|           0|
|25.TA.1-217-j19-1...|1-217-j19-1|  TA+b

In [46]:
%%spark
trips_table_filtered = trips_table_filtered.select("route_id", "service_id", "trip_id", "trip_headsign", "trip_short_name", "direction_id")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [47]:
%%spark
trips_table_filtered.write.mode('overwrite').option("delimiter", ";").option("header","true").format("csv").save("/group/five-guys/trips")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### Footpath table

In [49]:
%%spark

#add attributes lat, lon to stops_table
stops_table_expanded = stops_table.join(all_stops, 'stop_id', 'inner')
#create all possible pairs between stops
stops_table_expanded_a = stops_table_expanded.select([F.col(c).alias("a_"+c) for c in stops_table_expanded.columns])
stops_table_expanded_b = stops_table_expanded.select([F.col(c).alias("b_"+c) for c in stops_table_expanded.columns])
cartesian_product = stops_table_expanded_a.crossJoin(stops_table_expanded_b)
#compute distance for all pairs
stops_table_distance = cartesian_product.withColumn("a", (
        F.pow(F.sin(F.radians(F.col("a_stop_lat") - F.col("b_stop_lat")) / 2), 2) +
        F.cos(F.radians(F.col("b_stop_lat"))) * F.cos(F.radians(F.col("a_stop_lat"))) *
        F.pow(F.sin(F.radians(F.col("a_stop_lon") - F.col("b_stop_lon")) / 2), 2)
    )).withColumn("distance", F.atan2(F.sqrt(F.col("a")), F.sqrt(-F.col("a") + 1)) * 12742000)
#filter pair with distance shorter than 500m
stops_table_distance = stops_table_distance.filter(stops_table_distance.distance <= 500)
#compute time considering walking speed of 50m/1min
#time is in seconds
stops_table_time = stops_table_distance.withColumn('time', F.round((stops_table_distance['distance']/F.lit(50))*F.lit(60), 0).cast('int'))
footpath_table = stops_table_time.select(stops_table_time.a_stop_id.alias('departure_id'), stops_table_time.b_stop_id.alias('arrival_id'), stops_table_time.time)
#remove pairs where arrival == departure
footpath_table = footpath_table.filter(footpath_table.departure_id != footpath_table.arrival_id)
footpath_table.show(10)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------+-----------+----+
|departure_id| arrival_id|time|
+------------+-----------+----+
| 8503306:0:2|    8590541| 528|
|     8506895|    8573228| 402|
|     8591190|    8591170| 311|
|     8591284|    8591116| 479|
|     8591284|    8591274| 327|
|     8503078|    8591903| 471|
|     8589111|    8590655| 583|
|     8589111|    8594182| 293|
| 8503306:0:2|8503306:0:1|   0|
|     8589111|    8502208| 351|
+------------+-----------+----+
only showing top 10 rows

In [50]:
%%spark
footpath_table.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- departure_id: string (nullable = true)
 |-- arrival_id: string (nullable = true)
 |-- time: integer (nullable = true)

In [ ]:
%%spark
footpath_table.write.mode('overwrite').option("delimiter", ";").option("header","true").format("csv").save("/group/five-guys/footpath")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
#transform list of tuple into dictionary of dictionaries
#footpath_map = footpath_table.groupby('arrival_id')[['departure_id', 'time']].apply(lambda g: g.values.tolist()).to_dict()
#footpath_map = {k: dict(v) for k, v in footpath_map.items()}
#convert and save the dictionary to pickle
#pickle.dump(footpath_map, open(path_data+"footpath_map.pickle", "wb"))

In [ ]:
#!git lfs track "../data/footpath_map.pickle"
#!git add .gitattributes
#!git commit -m "footpath_map.pickle"
#!git push

## Confidence

### Import data

In [53]:
%%spark
#import filtered stop_ids
stops_table = spark.read.option("header",True).options(delimiter=';').csv("/group/five-guys/stops_table")
stops_table = stops_table.select(stops_table.stop_id)
stops_table.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- stop_id: string (nullable = true)

In [54]:
%%spark
actual_data = spark.read.orc('/data/sbb/orc/istdaten')
#actual_data.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [55]:
%%spark
#subsample
actual_data = actual_data.sample(0.001, seed=0)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### Rename columns

Columns are renamed in English as follows:
- `betriebstag` to DATE
- `produkt_id` to PRODUCT_ID
- `fahrt_bezeichner` to TRIP_ID
- `bpuic` to STOP_ID
- `haltestellen_name` to STOP_NAME
- `ankunftszeit` to ARRIVAL_TIME
- `an_prognose_status` to ARRIVAL_TIME_STATUS
- `an_prognose` to ACTUAL_ARRIVAL_TIME
- `abfahrtszeit` to DEPARTURE_TIME
- `ab_prognose_status` to DEPARTURE_TIME_STATUS
- `ab_prognose` to ACTUAL_DEPARTURE_TIME

In [56]:
%%spark

#rename columns in english
actual_data = actual_data.select(actual_data.betriebstag.alias('DATE'),\
                                 actual_data.produkt_id.alias('PRODUCT_ID'),\
                                 actual_data.bpuic.alias('STOP_ID'),\
                                 actual_data.haltestellen_name.alias('STOP_NAME'),\
                                 actual_data.fahrt_bezeichner.alias('TRIP_ID'),\
                                 actual_data.ankunftszeit.alias('ARRIVAL_TIME'),\
                                 actual_data.an_prognose_status.alias('ARRIVAL_TIME_STATUS'),\
                                 actual_data.an_prognose.alias('ACTUAL_ARRIVAL_TIME'),\
                                 actual_data.abfahrtszeit.alias('DEPARTURE_TIME'),\
                                 actual_data.ab_prognose_status.alias('DEPARTURE_TIME_STATUS'),
                                 actual_data.ab_prognose.alias('ACTUAL_DEPARTURE_TIME'))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### Filter stop_id

In [57]:
%%spark
#use only stop_ids from stops_table
tmp = actual_data.join(stops_table, (actual_data['STOP_ID'] == stops_table['stop_id']), how='inner')
actual_data = tmp.select(actual_data.DATE,\
                                 actual_data.PRODUCT_ID,\
                                 actual_data.STOP_ID,\
                                 actual_data.STOP_NAME,\
                                 actual_data.TRIP_ID,\
                                 actual_data.ARRIVAL_TIME,\
                                 actual_data.ARRIVAL_TIME_STATUS,\
                                 actual_data.ACTUAL_ARRIVAL_TIME,\
                                 actual_data.DEPARTURE_TIME,\
                                 actual_data.DEPARTURE_TIME_STATUS,
                                 actual_data.ACTUAL_DEPARTURE_TIME
                        )

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### Prepare table

In [58]:
%%spark
import pyspark.sql.functions as F
# F.date_format(F.to_timestamp(df_used.timestamp_s), 'yyyy-MM-dd HH:mm:ss').alias('date'))
df = actual_data.withColumn('DATE', F.date_format(F.to_timestamp(actual_data.DATE, 'dd.MM.yyyy'), 'yyyy-MM-dd'))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [59]:
%%spark
import pyspark.sql.functions as F

df = df.withColumn('DAY_OF_WEEK', ((F.dayofweek(df['DATE'])+5)%7)+1)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [60]:
%%spark
import pyspark.sql.functions as F

#remove entries with ARRIVAL_TIME_STATUS and DEPARTURE_TIME_STATUS unknown
df = df.filter(df['ARRIVAL_TIME_STATUS'] != 'UNBEKANNT')
df = df.filter(df['DEPARTURE_TIME_STATUS'] != 'UNBEKANNT')

#remove entries with ARRIVAL_TIME_STATUS and DEPARTURE_TIME_STATUS is empty
df = df.filter(df['ARRIVAL_TIME_STATUS'] != '')
df = df.filter(df['DEPARTURE_TIME_STATUS'] != '')

#remove entries with PRODUCT_ID empty
df = df.filter(df['PRODUCT_ID'] != '')

#remove entries with STOP_ID empty
df = df.filter(df['STOP_ID'] != '')

#remove entries with STOP_NAME empty
df = df.filter(df['STOP_NAME'] != '')

#remove entries with TRIP_ID empty
df = df.filter(df['TRIP_ID'] != '')

#transform to unix_timestamp
df = df.withColumn('ARRIVAL_TIME', F.unix_timestamp(df.ARRIVAL_TIME, 'dd.MM.yyyy HH:mm'))
df = df.withColumn('ACTUAL_ARRIVAL_TIME', F.unix_timestamp(df.ACTUAL_ARRIVAL_TIME, 'dd.MM.yyyy HH:mm'))
df = df.withColumn('DEPARTURE_TIME', F.unix_timestamp(df.DEPARTURE_TIME, 'dd.MM.yyyy HH:mm'))
df = df.withColumn('ACTUAL_DEPARTURE_TIME', F.unix_timestamp(df.ACTUAL_DEPARTURE_TIME, 'dd.MM.yyyy HH:mm'))

#fill null with 0 
#When the stop is the start or end of a journey, the corresponding columns will be empty (ANKUNFTSZEIT/ABFAHRTSZEIT)
df = df.na.fill({'ARRIVAL_TIME' : 0, 'ACTUAL_ARRIVAL_TIME' : 0, 'DEPARTURE_TIME' : 0, 'ACTUAL_DEPARTURE_TIME' : 0})

#compute delay
df = df.withColumn('ARRIVAL_DELAY', F.when((df['ACTUAL_ARRIVAL_TIME']-df['ARRIVAL_TIME'] )< 0, 0).otherwise(df['ACTUAL_ARRIVAL_TIME']-df['ARRIVAL_TIME']))
df = df.withColumn('DEPARTURE_DELAY', F.when((df['ACTUAL_DEPARTURE_TIME']-df['DEPARTURE_TIME'] )< 0, 0).otherwise(df['ACTUAL_DEPARTURE_TIME']-df['DEPARTURE_TIME']))

#convert seconds in minutes
df = df.withColumn('ARRIVAL_DELAY', F.round(df['ARRIVAL_DELAY']/60))
df = df.withColumn('DEPARTURE_DELAY', F.round(df['DEPARTURE_DELAY']/60))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [61]:
%%spark

#define HOUR_ARRIVAL
df = df.withColumn('HOUR_ARRIVAL', F.hour(F.from_unixtime(df['ARRIVAL_TIME'])))
df = df.withColumn('HOUR_DEPARTURE', F.hour(F.from_unixtime(df['DEPARTURE_TIME'])))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### Filter data based on simplifying assumptions

We only consider journeys at reasonable hours of the day, and on a typical business day.
- `day`: from Monday to Friday
- `hours`: first departure at 9:00, last arrival at 17

In [62]:
%%spark

#business day
df = df.filter(df['DAY_OF_WEEK'] <6)
#reasonable hours of the day
df = df.filter(F.hour(F.from_unixtime(df['DEPARTURE_TIME']))>=9)
df = df.filter(F.hour(F.from_unixtime(df['ARRIVAL_TIME']))<=17)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [63]:
%%spark

df.cache()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[DATE: string, PRODUCT_ID: string, STOP_ID: string, STOP_NAME: string, TRIP_ID: string, ARRIVAL_TIME: bigint, ARRIVAL_TIME_STATUS: string, ACTUAL_ARRIVAL_TIME: bigint, DEPARTURE_TIME: bigint, DEPARTURE_TIME_STATUS: string, ACTUAL_DEPARTURE_TIME: bigint, DAY_OF_WEEK: int, ARRIVAL_DELAY: double, DEPARTURE_DELAY: double, HOUR_ARRIVAL: int, HOUR_DEPARTURE: int]

In [64]:
%%spark

#create all possible pairs between stops
df_A = df.select([F.col(c).alias("A_"+c) for c in df.columns])
df_B = df.select([F.col(c).alias("B_"+c) for c in df.columns])

df_1 = connections_table.join(df_A, connections_table.departure_id == df_A.A_STOP_ID, how='inner')
df_1 = df_1.join(df_B, (df_1.arrival_id == df_B.B_STOP_ID) & (df_1.A_DATE == df_B.B_DATE))

df_2 = df_1.select(df_1.A_DAY_OF_WEEK.alias('DAY_OF_WEEK'),\
                   df_1.A_STOP_ID.alias('DEPARTURE_ID'),\
                   df_1.B_STOP_ID.alias('ARRIVAL_ID'),\
                   df_1.B_HOUR_ARRIVAL.alias('HOUR_ARRIVAL'),\
                   df_1.B_ARRIVAL_DELAY.alias('ARRIVAL_DELAY')
                  )


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [65]:
# %%spark

# #create all possible pairs between stops
# df_A = df.select([F.col(c).alias("A_"+c) for c in df.columns])
# df_B = df.select([F.col(c).alias("B_"+c) for c in df.columns])
# cp = df_A.crossJoin(df_B)
# #keep only pairs (A_STOP_ID, B_STOP_ID) that are in connections_table

# df_1 = cp.join(connections_table, (connections_table.departure_id == cp.A_STOP_ID) & (connections_table.arrival_id == cp.B_STOP_ID), how='inner')
# #select only interesting fields
# df_2 = df_1.select(df_1.A_DAY_OF_WEEK.alias('DAY_OF_WEEK'),\
#                    df_1.A_STOP_ID.alias('DEPARTURE_ID'),\
#                    df_1.B_STOP_ID.alias('ARRIVAL_ID'),\
#                    df_1.B_HOUR_ARRIVAL.alias('HOUR_ARRIVAL'),\
#                    df_1.trip_id.alias('TRIP_ID'),\
#                    df_1.B_ARRIVAL_DELAY.alias('ARRIVAL_DELAY')
#                   )
# df_2.cache()

### Obtain delay cumulative distribution

In [66]:
%%spark
from pyspark.sql import Window

#compute cumulative distribution of delay over groupBy 'DAY_OF_WEEK', 'PRODUCT_ID', 'HOUR_ARRIVAL' 
cumulative_function_window = Window.partitionBy('DAY_OF_WEEK', 'DEPARTURE_ID', 'ARRIVAL_ID', 'TRIP_ID').orderBy('ARRIVAL_DELAY').rangeBetween(Window.unboundedPreceding, 0)
count_window = Window.partitionBy('DAY_OF_WEEK', 'DEPARTURE_ID', 'ARRIVAL_ID', 'TRIP_ID')

df2 = df_2.groupBy(df_2['DAY_OF_WEEK'], df_2['DEPARTURE_ID'], df_2['ARRIVAL_ID'], df_2['ARRIVAL_DELAY'], df_2['TRIP_ID']).count()
df2 = df2.withColumn("TOTAL_SUM", F.sum("count").over(count_window))
df2 = df2.withColumn("PARTIAL_SUM", F.sum("count").over(cumulative_function_window))
df2 = df2.withColumn("CUMULATIVE", F.sum("count").over(cumulative_function_window)/F.sum("count").over(count_window))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [67]:
%%spark

#reorder columns
df2 = df2.select(df2["TRIP_ID"], df2["DEPARTURE_ID"], df2["ARRIVAL_ID"], df2["DAY_OF_WEEK"], df2["ARRIVAL_DELAY"].alias("MAX_ARRIVAL_DELAY"), df2["CUMULATIVE"])
#df2 = df2.orderBy(df2["TRIP_ID"], df2["DEPARTURE_ID"], df2["ARRIVAL_ID"], df2["DAY_OF_WEEK"], df2["HOUR_ARRIVAL"], df2["MAX_ARRIVAL_DELAY"])

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Split based on day

In [69]:
%%spark
df2.show(10)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+------------+----------+-----------+-----------------+------------------+
|             TRIP_ID|DEPARTURE_ID|ARRIVAL_ID|DAY_OF_WEEK|MAX_ARRIVAL_DELAY|        CUMULATIVE|
+--------------------+------------+----------+-----------+-----------------+------------------+
|229.TA.26-301-j19...|     8500926|   8590737|          1|              0.0|               0.5|
|229.TA.26-301-j19...|     8500926|   8590737|          1|              1.0|               1.0|
|248.TA.26-301-j19...|     8500926|   8590737|          1|              0.0|               0.5|
|248.TA.26-301-j19...|     8500926|   8590737|          1|              1.0|               1.0|
|25.TA.30-56-Y-j19...|     8502209|   8503202|          1|              0.0|0.7142857142857143|
|25.TA.30-56-Y-j19...|     8502209|   8503202|          1|              1.0|               1.0|
|45.TA.1-17-A-j19-...|     8502274|   8502275|          1|              0.0|               1.0|
|379.TA.26-184-j19...|     8502495|   85

In [70]:
%%spark
#all

df2.write.mode('overwrite').option("delimiter", ";").option("header","true").format("csv").save("/group/five-guys/confidence")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [71]:
%%spark
#monday

df2_monday = df2.filter(df2.DAY_OF_WEEK == 1)
df2_monday.write.mode('overwrite').option("delimiter", ";").option("header","true").format("csv").save("/group/five-guys/confidence_monday")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
%%spark
#tuesday

df2_tuesday = df2.filter(df2.DAY_OF_WEEK == 2)
df2_tuesday.write.mode('overwrite').option("delimiter", ";").option("header","true").format("csv").save("/group/five-guys/confidence_tuesday")

In [ ]:
%%spark
#wednesday

df2_wednesday = df2.filter(df2.DAY_OF_WEEK == 3)
df2_wednesday.write.mode('overwrite').option("delimiter", ";").option("header","true").format("csv").save("/group/five-guys/confidence_wednesday")

In [ ]:
%%spark
#thursday

df2_thursday = df2.filter(df2.DAY_OF_WEEK == 4)
df2_thursday.write.mode('overwrite').option("delimiter", ";").option("header","true").format("csv").save("/group/five-guys/confidence_thursday")

In [ ]:
%%spark
#friday

df2_friday = df2.filter(df2.DAY_OF_WEEK == 5)
df2_friday.write.mode('overwrite').option("delimiter", ";").option("header","true").format("csv").save("/group/five-guys/confidence_friday")